In [37]:
import numpy as np
import pandas as pd

In [46]:
def lzc(val, bits=32):
    """Count leading zeroes."""
    cnt = 0
    for i in range(0, bits):
        if val & (1 << (bits - 1 - i)) != 0:
            break
        cnt += 1
    return cnt
lzcu = np.frompyfunc(lzc, 2, 1)

In [130]:
def abs_diff(truth, pred):
    assert truth.size == pred.size, "Different sizes"
    result = np.zeros(truth.size).astype(truth.dtype)
    sign = np.ones(truth.size).astype(bool) * False
    for i in range(truth.size):
        low_shot = truth[i] > pred[i]
        result[i] = truth[i] - pred[i] if low_shot else pred[i] - truth[i]
        sign[i] = low_shot
    return sign, result

In [131]:
truth = np.fromfile("test.npf", "int64")

In [132]:
df = pd.DataFrame({'truth':truth})

In [133]:
df['prediction'] = df.truth.shift()
df.loc[0,'prediction'] = 0
df = df.astype('uint32')

In [134]:
df['xor'] = df.truth ^ df.prediction
df['low_shot'], df['diff'] = abs_diff(df.truth, df.prediction)
df['lzc_xor'] = lzcu(df.loc[:,'xor'], 32)
df['lzc_diff'] = lzcu(df.loc[:,'diff'], 32)

In [135]:
df.head()

,truth,prediction,xor,low_shot,diff,lzc_xor,lzc_diff
0,1228193487,0,1228193487,True,1228193487,1,1
1,1228175479,1228193487,50872,False,18008,16,17
2,1228178323,1228175479,64484,True,2844,16,20
3,1228204314,1228178323,27273,True,25991,17,17
4,1228217687,1228204314,128077,True,13373,15,18


In [136]:
all_cases = df.index.size
diff_better = df.loc[:,'xor'] >= df.loc[:,'diff']
diff_bits_better = lzcu(df.loc[:,'xor'], 32) < lzcu(df.loc[:,'diff'], 32)
diff_bits_better_2bits = lzcu(df.loc[:,'xor'], 32) + 2 <= lzcu(df.loc[:,'diff'], 32) 
all_cases, diff_better.sum(), diff_bits_better.sum(), diff_bits_better_2bits.sum()

(1000, 1000, 721, 349)

In [137]:
df.lzc_diff.sum(), df.lzc_xor.sum()
# diff_lzc - sign_bits, xor_lzc + first_bit

(19388, 18049)

In [138]:
df[df.lzc_xor > df.lzc_diff].size

0

- DIFF is often smaller than XOR regarding absolute representation or bit.
- 

In [140]:
df.low_shot

0       True
1      False
2       True
3       True
4       True
5      False
6      False
7       True
8       True
9       True
10      True
11      True
12     False
13      True
14      True
15     False
16     False
17     False
18     False
19      True
20      True
21     False
22     False
23     False
24     False
25     False
26      True
27     False
28     False
29      True
       ...  
970     True
971     True
972    False
973     True
974     True
975    False
976    False
977     True
978    False
979     True
980     True
981     True
982     True
983     True
984    False
985    False
986    False
987     True
988    False
989    False
990    False
991    False
992    False
993     True
994    False
995    False
996    False
997    False
998    False
999    False
Name: low_shot, Length: 1000, dtype: bool